In [1]:
from pyspark.sql import SparkSession
from functools import reduce

spark = SparkSession.builder.master("local").appName("RankTimeSeries").config("spark.some.config.option", "some-value").getOrCreate()
# 11809233

In [12]:
from pyspark.sql.functions import unix_timestamp,rank, desc
from pyspark.sql.window import Window

df = spark.read.format("csv").option("header", "true").load('data/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv')

Dates = df.select('Summons Number', unix_timestamp('Issue Date', 'MM/dd/yyyy').alias('date')).fillna(0)
oldColumns = Dates.schema.names
newColumns = ["SN", "date"]
Dates = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), Dates)
Dates.printSchema()
Dates.show()
Dates.createOrReplaceTempView("initial")
Ranked = spark.sql("""
    SELECT i.SN, i.date, ROW_NUMBER() OVER (ORDER BY i.date DESC) AS position
    FROM initial as i
    """)
#Ranked = Dates.withColumn("position", rank().over(window))
#Ranked.coalesce(1)

#Ranked.write.format('com.databricks.spark.csv').save('data/ranked',mode='overwrite')



root
 |-- SN: string (nullable = true)
 |-- date: long (nullable = true)

+----------+----------+
|        SN|      date|
+----------+----------+
|8002531292|1412136000|
|8015318440|1425618000|
|7611181981|1406520000|
|7445908067|1428897600|
|7037692864|1432008000|
|7704791394|1416459600|
|7541599104|1406865600|
|7443513951|1405742400|
|7426321423|1411444800|
|8017159560|1421730000|
|8017159560|1421730000|
|7002571382|1424149200|
|7939966900|1430798400|
|7704167637|1413864000|
|7095480941|1408593600|
|7831181295|1407384000|
|8007006229|1409025600|
|1369109532|1403496000|
|7580650400|1411531200|
|7420795630|1412395200|
+----------+----------+
only showing top 20 rows



DataFrame[SN: string, date: bigint, position: int]

In [15]:
Ranked.show()


+----------+----------+--------+
|        SN|      date|position|
+----------+----------+--------+
|7713884014|1435636800|       1|
|1383240346|1435636800|       2|
|8027900840|1435636800|       3|
|5087037630|1435636800|       4|
|7575453805|1435636800|       5|
|5087045698|1435636800|       6|
|7099890669|1435636800|       7|
|4004752449|1435636800|       8|
|7249051659|1435636800|       9|
|5087040494|1435636800|      10|
|8043760111|1435636800|      11|
|1385767900|1435636800|      12|
|1384524812|1435636800|      13|
|5087044463|1435636800|      14|
|8025982324|1435636800|      15|
|1379731926|1435636800|      16|
|7582369506|1435636800|      17|
|1385282368|1435636800|      18|
|7298797645|1435636800|      19|
|4004751524|1435636800|      20|
+----------+----------+--------+
only showing top 20 rows



In [16]:
Ranked.write.csv("csv")


Py4JJavaError: An error occurred while calling o265.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:196)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperation

In [ ]:
df = spark.read.format("csv").option("header", "true").load('data/nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2015.csv')

Dates = df.select('Summons Number')
window = Window.orderBy(desc('Summons Number'))
Ranked = Dates.withColumn("position", rank().over(window))
print(Ranked.count())

